In [1]:
##------------------Import Libraries --------------------------##
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
##------------------Read Data----------------------------------##
file_path = 'SIPP_analysis_data.xlsx'
data = pd.read_excel(file_path)

In [3]:
data.head()

,unique_id,ssuid,repwgt0,repwgt1,repwgt2,repwgt3,repwgt4,repwgt5,repwgt6,repwgt7,...,unsecured_debt,unable_pay,mortgage_payment,asset_values,bachelors,years_since_movein,unemployed,out_lf,born_abroad,likely_retired
0,1,11428546521,7426.116541,3272.646846,2984.807011,4144.434128,3603.779767,12749.500402,3534.087316,3677.640142,...,0,0,1390,430510,0,-140.416672,0,0,1,0
1,2,11428577022,9260.630659,17036.969529,13129.978967,13180.834760,15801.090740,4445.420139,12091.404756,12781.709130,...,40300,1,1300,41732,0,-163.166672,0,0,0,0
2,3,11481674622,4837.305089,8031.548547,8130.963960,7984.299255,7644.212702,2273.861069,7101.412235,7670.058779,...,20000,0,500,169050,1,-130.666672,0,1,0,1
3,4,12889229422,9638.013075,16219.670390,13163.080201,12818.263387,12173.747482,13922.769296,5538.784662,14840.845358,...,0,0,0,6880,0,-165.333328,0,0,0,0
4,5,13309398822,5190.990802,8106.156293,7452.235473,9004.767596,2426.520981,2555.204602,7252.986835,9276.875285,...,15000,0,1200,400,0,NaN,0,1,0,0


In [4]:
data.isnull().sum()

unique_id                0
ssuid                    0
repwgt0                  0
repwgt1                  0
repwgt2                  0
                      ... 
years_since_movein    2747
unemployed               0
out_lf                   0
born_abroad              0
likely_retired           0
Length: 324, dtype: int64

In [5]:
data.dtypes

unique_id               int64
ssuid                   int64
repwgt0               float64
repwgt1               float64
repwgt2               float64
                       ...   
years_since_movein    float64
unemployed              int64
out_lf                  int64
born_abroad             int64
likely_retired          int64
Length: 324, dtype: object

In [6]:
data = data.drop('years_since_movein', axis=1)

In [7]:
data.columns

Index(['unique_id', 'ssuid', 'repwgt0', 'repwgt1', 'repwgt2', 'repwgt3',
       'repwgt4', 'repwgt5', 'repwgt6', 'repwgt7',
       ...
       'secured_debt', 'unsecured_debt', 'unable_pay', 'mortgage_payment',
       'asset_values', 'bachelors', 'unemployed', 'out_lf', 'born_abroad',
       'likely_retired'],
      dtype='object', length=323)

In [8]:
## Display all columns
pd.set_option('display.max_columns', None)

In [ ]:
##---------code not necessary for now--------------##
#print(data.columns.tolist())

['unique_id', 'ssuid', 'repwgt0', 'repwgt1', 'repwgt2', 'repwgt3', 'repwgt4', 'repwgt5', 'repwgt6', 'repwgt7', 'repwgt8', 'repwgt9', 'repwgt10', 'repwgt11', 'repwgt12', 'repwgt13', 'repwgt14', 'repwgt15', 'repwgt16', 'repwgt17', 'repwgt18', 'repwgt19', 'repwgt20', 'repwgt21', 'repwgt22', 'repwgt23', 'repwgt24', 'repwgt25', 'repwgt26', 'repwgt27', 'repwgt28', 'repwgt29', 'repwgt30', 'repwgt31', 'repwgt32', 'repwgt33', 'repwgt34', 'repwgt35', 'repwgt36', 'repwgt37', 'repwgt38', 'repwgt39', 'repwgt40', 'repwgt41', 'repwgt42', 'repwgt43', 'repwgt44', 'repwgt45', 'repwgt46', 'repwgt47', 'repwgt48', 'repwgt49', 'repwgt50', 'repwgt51', 'repwgt52', 'repwgt53', 'repwgt54', 'repwgt55', 'repwgt56', 'repwgt57', 'repwgt58', 'repwgt59', 'repwgt60', 'repwgt61', 'repwgt62', 'repwgt63', 'repwgt64', 'repwgt65', 'repwgt66', 'repwgt67', 'repwgt68', 'repwgt69', 'repwgt70', 'repwgt71', 'repwgt72', 'repwgt73', 'repwgt74', 'repwgt75', 'repwgt76', 'repwgt77', 'repwgt78', 'repwgt79', 'repwgt80', 'repwgt81', 're

In [8]:
num_column = data.select_dtypes(include=['int64', 'float64'])
cat_column = data.select_dtypes(include='object')

#### Data Preparation

In [9]:
###--------------------find replicate weights----------------------###
repl_weights = [col for col in num_column.columns if 'repwgt' in col]

In [ ]:
###--------------------find features-----------------------------###
# Assuming that our target variable is 'asset_values'
# this is only to showcase how to model the replicate weights
target = 'asset_values'
features =  [col for col in num_column.columns if col not in repl_weights and col != target and 'area_state' not in col]


In [11]:
###-------------------Primary Weights---------------------------###
primary_weight = 'wpfinwgt'

In [12]:
num_column.isnull().sum()

unique_id         0
ssuid             0
repwgt0           0
repwgt1           0
repwgt2           0
                 ..
bachelors         0
unemployed        0
out_lf            0
born_abroad       0
likely_retired    0
Length: 317, dtype: int64

In [13]:
##------Show columns with only NaN---------###
nan_columns = num_column.columns[num_column.isnull().any()]
num_column_nan = num_column[nan_columns]

In [14]:
num_column.head()

,unique_id,ssuid,repwgt0,repwgt1,repwgt2,repwgt3,repwgt4,repwgt5,repwgt6,repwgt7,...,secured_debt,unsecured_debt,unable_pay,mortgage_payment,asset_values,bachelors,unemployed,out_lf,born_abroad,likely_retired
0,1,11428546521,7426.116541,3272.646846,2984.807011,4144.434128,3603.779767,12749.500402,3534.087316,3677.640142,...,409000,0,0,1390,430510,0,0,0,1,0
1,2,11428577022,9260.630659,17036.969529,13129.978967,13180.834760,15801.090740,4445.420139,12091.404756,12781.709130,...,0,40300,1,1300,41732,0,0,0,0,0
2,3,11481674622,4837.305089,8031.548547,8130.963960,7984.299255,7644.212702,2273.861069,7101.412235,7670.058779,...,0,20000,0,500,169050,1,0,1,0,1
3,4,12889229422,9638.013075,16219.670390,13163.080201,12818.263387,12173.747482,13922.769296,5538.784662,14840.845358,...,0,0,0,0,6880,0,0,0,0,0
4,5,13309398822,5190.990802,8106.156293,7452.235473,9004.767596,2426.520981,2555.204602,7252.986835,9276.875285,...,0,15000,0,1200,400,0,0,1,0,0


In [15]:
num_column_nan

,area_state1,area_state2,area_state4,area_state5,area_state6,area_state8,area_state9,area_state10,area_state11,area_state12,...,area_state46,area_state47,area_state48,area_state49,area_state50,area_state51,area_state53,area_state54,area_state55,area_state56
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17433,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17434,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17435,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17436,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
##---------Exclude the columns with NaN values----------------##
num_column_clean = num_column.dropna(axis=1, inplace=True)

In [16]:
num_column.isnull().sum()

unique_id         0
ssuid             0
repwgt0           0
repwgt1           0
repwgt2           0
                 ..
bachelors         0
unemployed        0
out_lf            0
born_abroad       0
likely_retired    0
Length: 317, dtype: int64

In [17]:
###-----------------Fit data on Primary Weight-----------------###

X = num_column[features]
y = num_column[target]
sample_weight = num_column[primary_weight]

###------------------Split Data-------------------------------###
X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(X, y, sample_weight, test_size=0.3, random_state=42)

model = LinearRegression()
"Extra Coding process needed if doing cross-validation, or other hyperparametization"
"Might have to make process changes based on a different model."
model.fit(X_train, y_train, sample_weight=weights_train)

# Evaluate the model
# predict on the test set
baseline_predictions = model.predict(X_test)
baseline_mse = mean_squared_error(y_test, baseline_predictions)

print(f'Baseline Model MSE: {baseline_mse}')
print(f'Model Coefficients: {model.coef_}')
print(f'Intercept: {model.intercept_}')

Baseline Model MSE: 1.3619245732091876e-17
Model Coefficients: [ 8.36045858e-15  8.27180613e-25  5.24943158e-14  1.00000000e+00
  0.00000000e+00 -8.62624972e-16  5.07694077e-13 -5.17836841e-12
  3.22429557e-12 -1.26186026e-11  1.29309160e-10  4.98543644e-16
  3.25362982e-16  2.37011237e-16  1.41619216e-15  1.00000000e+00
  1.00000000e+00  4.59098527e-12 -3.15066900e-17  2.11580761e-11
  7.79448002e-12 -1.37981622e-11 -2.91155665e-11 -2.45205092e-11]
Intercept: 2.3283064365386963e-10


In [ ]:
X

,unique_id,ssuid,wpfinwgt,net_worth,area_US,hh_inc,age,female,hh_size,married,...,credit_card_debt,secured_debt,unsecured_debt,unable_pay,mortgage_payment,bachelors,unemployed,out_lf,born_abroad,likely_retired
0,1,11428546521,7426.116541,21510,1,110326,64,0,3,0,...,0,409000,0,0,1390,0,0,0,1,0
1,2,11428577022,9260.630659,1432,1,120422,32,1,5,1,...,0,0,40300,1,1300,0,0,0,0,0
2,3,11481674622,4837.305089,149050,1,0,75,0,3,1,...,20000,0,20000,0,500,1,0,1,0,1
3,4,12889229422,9638.013075,6880,1,3644,21,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,13309398822,5190.990802,-14600,1,9168,21,0,1,0,...,0,0,15000,0,1200,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17433,17434,88199599536620,5198.561179,2820,1,0,64,0,1,0,...,0,0,0,0,633,0,0,1,1,0
17434,17435,88199599536622,3708.536432,4470,1,65597,49,1,4,0,...,0,26000,0,0,1380,0,0,0,1,0
17435,17436,88199599547022,6117.100620,484616,1,118740,41,1,5,1,...,30000,479000,80000,0,3900,1,0,0,0,0
17436,17437,88199599596020,7181.381218,0,1,0,89,1,2,0,...,0,0,0,0,860,0,0,1,1,1


In [20]:
### Check Shapes of X and repl_weights
print(f"X shape: {X.shape}")
print(f"repl_weights: {repl_weights}")

X shape: (17438, 24)
repl_weights: ['repwgt0', 'repwgt1', 'repwgt2', 'repwgt3', 'repwgt4', 'repwgt5', 'repwgt6', 'repwgt7', 'repwgt8', 'repwgt9', 'repwgt10', 'repwgt11', 'repwgt12', 'repwgt13', 'repwgt14', 'repwgt15', 'repwgt16', 'repwgt17', 'repwgt18', 'repwgt19', 'repwgt20', 'repwgt21', 'repwgt22', 'repwgt23', 'repwgt24', 'repwgt25', 'repwgt26', 'repwgt27', 'repwgt28', 'repwgt29', 'repwgt30', 'repwgt31', 'repwgt32', 'repwgt33', 'repwgt34', 'repwgt35', 'repwgt36', 'repwgt37', 'repwgt38', 'repwgt39', 'repwgt40', 'repwgt41', 'repwgt42', 'repwgt43', 'repwgt44', 'repwgt45', 'repwgt46', 'repwgt47', 'repwgt48', 'repwgt49', 'repwgt50', 'repwgt51', 'repwgt52', 'repwgt53', 'repwgt54', 'repwgt55', 'repwgt56', 'repwgt57', 'repwgt58', 'repwgt59', 'repwgt60', 'repwgt61', 'repwgt62', 'repwgt63', 'repwgt64', 'repwgt65', 'repwgt66', 'repwgt67', 'repwgt68', 'repwgt69', 'repwgt70', 'repwgt71', 'repwgt72', 'repwgt73', 'repwgt74', 'repwgt75', 'repwgt76', 'repwgt77', 'repwgt78', 'repwgt79', 'repwgt80', 'r

In [ ]:
## Fix Shape Mismatch


In [19]:
X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(X, y, repl_weights, test_size=0.3, random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [17438, 17438, 241]

In [40]:
X_train.shape

(12206, 24)

In [41]:
num_column.shape

(17438, 266)

In [42]:
12206/17438

0.6999655923844478

In [22]:
###-----------------Fit on Replicate Weights-----------------------###
repl_prod = []

for weight in repl_weights:
    weight_sample = num_column[weight]
    # check for mismatch between weight and sample shape
    if weight_sample.shape[0] != X_train.shape[0]:
        print(f'Weight sample for {weight} has a mismatch in the number of samples')
        weight_sample = weight_sample[:X_train.shape[0]]
    model.fit(X_train, y_train, sample_weight=weight_sample)
    predictions = model.predict(X_test)
    repl_prod.append(predictions)

# convert to dataframe
repl_prod = pd.DataFrame(repl_prod).T

Weight sample for repwgt0 has a mismatch in the number of samples
Weight sample for repwgt1 has a mismatch in the number of samples
Weight sample for repwgt2 has a mismatch in the number of samples
Weight sample for repwgt3 has a mismatch in the number of samples
Weight sample for repwgt4 has a mismatch in the number of samples
Weight sample for repwgt5 has a mismatch in the number of samples
Weight sample for repwgt6 has a mismatch in the number of samples
Weight sample for repwgt7 has a mismatch in the number of samples
Weight sample for repwgt8 has a mismatch in the number of samples
Weight sample for repwgt9 has a mismatch in the number of samples
Weight sample for repwgt10 has a mismatch in the number of samples
Weight sample for repwgt11 has a mismatch in the number of samples
Weight sample for repwgt12 has a mismatch in the number of samples
Weight sample for repwgt13 has a mismatch in the number of samples
Weight sample for repwgt14 has a mismatch in the number of samples
Weigh

In [37]:
# calculate variance across replicate predictions
prediction_variance = repl_prod.var(axis=1)

# Average predictions across replicate models for final estimate.
final_predictions = repl_prod.mean(axis=1)

print(f'Average Predictions for replicate weights: \n{final_predictions}')

Average Predictions for replicate weights: 
0         392840.0
1         328500.0
2           1005.0
3         224860.0
4         246400.0
           ...    
5227      713301.0
5228    12522080.0
5229      355500.0
5230       30980.0
5231      149550.0
Length: 5232, dtype: float64


In [ ]:
repl_weights = ['repwgt0', 'repwgt1', 'repwgt2', 'repwgt3', 'repwgt4', 'repwgt5', 'repwgt6', 'repwgt7', 'repwgt8', 'repwgt9', 'repwgt10', 'repwgt11', 'repwgt12', 'repwgt13', 'repwgt14', 'repwgt15', 'repwgt16', 
            'repwgt17', 'repwgt18', 'repwgt19', 'repwgt20', 'repwgt21', 'repwgt22', 'repwgt23', 'repwgt24', 'repwgt25', 'repwgt26', 'repwgt27', 'repwgt28', 'repwgt29', 'repwgt30', 'repwgt31', 'repwgt32', 'repwgt33', 
            'repwgt34', 'repwgt35', 'repwgt36', 'repwgt37', 'repwgt38', 'repwgt39', 'repwgt40', 'repwgt41', 'repwgt42', 'repwgt43', 'repwgt44', 'repwgt45', 'repwgt46', 'repwgt47', 'repwgt48', 'repwgt49', 
            'repwgt50', 'repwgt51', 'repwgt52', 'repwgt53', 'repwgt54', 'repwgt55', 'repwgt56', 'repwgt57', 'repwgt58', 'repwgt59', 'repwgt60', 'repwgt61', 'repwgt62', 'repwgt63', 'repwgt64', 'repwgt65', 
            'repwgt66', 'repwgt67', 'repwgt68', 'repwgt69', 'repwgt70', 'repwgt71', 'repwgt72', 'repwgt73', 'repwgt74', 'repwgt75', 'repwgt76', 'repwgt77', 'repwgt78', 'repwgt79', 'repwgt80', 'repwgt81', 
            'repwgt82', 'repwgt83', 'repwgt84', 'repwgt85', 'repwgt86', 'repwgt87', 'repwgt88', 'repwgt89', 'repwgt90', 'repwgt91', 'repwgt92', 'repwgt93', 'repwgt94', 'repwgt95', 'repwgt96', 'repwgt97', 
            'repwgt98', 'repwgt99', 'repwgt100', 'repwgt101', 'repwgt102', 'repwgt103', 'repwgt104', 'repwgt105', 'repwgt106', 'repwgt107', 'repwgt108', 'repwgt109', 'repwgt110', 'repwgt111', 'repwgt112', 
            'repwgt113', 'repwgt114', 'repwgt115', 'repwgt116', 'repwgt117', 'repwgt118', 'repwgt119', 'repwgt120', 'repwgt121', 'repwgt122', 'repwgt123', 'repwgt124', 'repwgt125', 'repwgt126', 'repwgt127', 
            'repwgt128', 'repwgt129', 'repwgt130', 'repwgt131', 'repwgt132', 'repwgt133', 'repwgt134', 'repwgt135', 'repwgt136', 'repwgt137', 'repwgt138', 'repwgt139', 'repwgt140', 'repwgt141', 'repwgt142', 
            'repwgt143', 'repwgt144', 'repwgt145', 'repwgt146', 'repwgt147', 'repwgt148', 'repwgt149', 'repwgt150', 'repwgt151', 'repwgt152', 'repwgt153', 'repwgt154', 'repwgt155', 'repwgt156', 'repwgt157', 
            'repwgt158', 'repwgt159', 'repwgt160', 'repwgt161', 'repwgt162', 'repwgt163', 'repwgt164', 'repwgt165', 'repwgt166', 'repwgt167', 'repwgt168', 'repwgt169', 'repwgt170', 'repwgt171', 'repwgt172', 
            'repwgt173', 'repwgt174', 'repwgt175', 'repwgt176', 'repwgt177', 'repwgt178', 'repwgt179', 'repwgt180', 'repwgt181', 'repwgt182', 'repwgt183', 'repwgt184', 'repwgt185', 'repwgt186', 'repwgt187', 
            'repwgt188', 'repwgt189', 'repwgt190', 'repwgt191', 'repwgt192', 'repwgt193', 'repwgt194', 'repwgt195', 'repwgt196', 'repwgt197', 'repwgt198', 'repwgt199', 'repwgt200', 'repwgt201', 'repwgt202', 
            'repwgt203', 'repwgt204', 'repwgt205', 'repwgt206', 'repwgt207', 'repwgt208', 'repwgt209', 'repwgt210', 'repwgt211', 'repwgt212', 'repwgt213', 'repwgt214', 'repwgt215', 'repwgt216', 'repwgt217', 
            'repwgt218', 'repwgt219', 'repwgt220', 'repwgt221', 'repwgt222', 'repwgt223', 'repwgt224', 'repwgt225', 'repwgt226', 'repwgt227', 'repwgt228', 'repwgt229', 'repwgt230', 'repwgt231', 'repwgt232', 
            'repwgt233', 'repwgt234', 'repwgt235', 'repwgt236', 'repwgt237', 'repwgt238', 'repwgt239', 'repwgt240']

In [ ]:

features_ = ['net_worth', 'area_US', 'race_eth_det', 'race_det5', 'race_eth', 'race_eth3', 'race_det3', 'poc', 'hh_inc', 'age', 
            'female', 'hh_size', 'married', 'homeowner', 'home_value', 
               'home_debt', 'student_debt', 'credit_card_debt', 'secured_debt', 'unsecured_debt', 'unable_pay', 'mortgage_payment',
               'asset_values', 'bachelors', 'years_since_movein', 'unemployed', 'out_lf', 'born_abroad', 'likely_retired']

In [ ]:
###-----------Better way to Extract Columns-------------------------###
# Extract replicate weight columns
repl_weights = [col for col in data.columns if 'repwgt' in col]

# Extract feature columns
features =  [col for col in num_column.columns if col not in repl_weights and col != target and 'area_state' not in col]